# Given halo mass cut, blackhole mass cut, central or satellite => gives halo IDs based on accretion efficiency and luminosity

In [1]:
import numpy as np
import illustris_python as il

In [2]:
basePath = "../../../sims.TNG/TNG100-1/output/"
snap = 50                                               #z=1

## specific eddington ratio cut

In [3]:
# Constants
c = 299792458  # speed of light in m/s
mp = 1.67 * 1e-27  # kg
G = 6.674 * 1e-11  # N m^2 kg^-2
sigma_t = 6.652 * 1e-29  # m^2
h = 0.6774
e_r = 0.2

shalo = il.groupcat.loadSubhalos(basePath, snap, fields=['SubhaloGrNr', 'SubhaloBHMdot', 'SubhaloBHMass'])
halo = il.groupcat.loadHalos(basePath, snap, fields =['Group_M_Crit500', 'GroupFirstSub'])

Mhalo = halo['Group_M_Crit500'] * 1e10 / h  #Msun
Mdot = shalo["SubhaloBHMdot"] * 10.22       #Msun / yr
M = shalo["SubhaloBHMass"]* 1e10 / h        #Msun
subID = np.arange(len(Mdot))
phid = shalo['SubhaloGrNr']
phm = Mhalo[phid[subID]]

group_firstsub = halo['GroupFirstSub']
group_firstsub = group_firstsub[group_firstsub != -1]
halo_centralID = np.array(group_firstsub)

central = np.zeros(len(M))
central[halo_centralID] = 1

indices = np.where((M > 1e6) & (phm > 10**12) & (phm < 10**12.5) & (central == 1))

Mdot_filtered = Mdot[indices]
M_filtered = M[indices]
subID_filtered = subID[indices]

Mdot_edd = (4 * np.pi * G * M_filtered * mp) / (e_r * c * sigma_t) / 3.16887646e-8  # M_sun/yr

edd_ratio = Mdot_filtered / Mdot_edd

efficiency = 10 * edd_ratio * e_r
efficiency[edd_ratio > 0.1] = e_r /(1-e_r)
Lbol = efficiency * Mdot_filtered * (1.989e+30 / 3.154e+7) * c**2 * 1e+7         #ergs/s

threshold = 0.1
Lcut = 1e41

sa_agn = subID_filtered[np.where((edd_ratio > threshold) & (Lbol > Lcut))]
sa_non = subID_filtered[np.where((edd_ratio > threshold) & (Lbol <= Lcut))]
sn_agn = subID_filtered[np.where((edd_ratio <= threshold) & (Lbol > Lcut))]
sn_non = subID_filtered[np.where((edd_ratio <= threshold) & (Lbol <= Lcut))]


ha_agn = phid[sa_agn]
ha_non = phid[sa_non]
hn_agn = phid[sn_agn]
hn_non = phid[sn_non]

np.savez("accretor_agn50_hm3.npz", ha_agn = ha_agn, ha_non = ha_non, hn_agn = hn_agn, hn_non = hn_non)

# Print stats
total = len(subID_filtered)
print("Total halos:", total)
print("High accretor Central AGNs:", len(ha_agn))
print("High accretor Central Non-AGNs:", len(ha_non))
print("Non-accretor Central AGNs:", len(hn_agn))
print("Non-accretor Central Non-AGNs:", len(hn_non))

Total halos: 1020
High accretor Central AGNs: 41
High accretor Central Non-AGNs: 0
Non-accretor Central AGNs: 516
Non-accretor Central Non-AGNs: 463


## TNG prescribed categorization

In [4]:
# Constants
c = 299792458  # speed of light in m/s
mp = 1.67 * 1e-27  # kg
G = 6.674 * 1e-11  # N m^2 kg^-2
sigma_t = 6.652 * 1e-29  # m^2
h = 0.6774
e_r = 0.2

print("working")

shalo = il.groupcat.loadSubhalos(basePath, snap, fields=['SubhaloGrNr', 'SubhaloBHMdot', 'SubhaloBHMass'])
halo = il.groupcat.loadHalos(basePath, snap, fields =['Group_M_Crit500', 'GroupFirstSub'])

Mhalo = halo['Group_M_Crit500'] * 1e10 / h  #Msun
Mdot = shalo["SubhaloBHMdot"] * 10.22       #Msun / yr
M = shalo["SubhaloBHMass"]* 1e10 / h        #Msun
subID = np.arange(len(Mdot))
phid = shalo['SubhaloGrNr']
phm = Mhalo[phid[subID]]

group_firstsub = halo['GroupFirstSub']
group_firstsub = group_firstsub[group_firstsub != -1]
halo_centralID = np.array(group_firstsub)

central = np.zeros(len(M))
central[halo_centralID] = 1

indices = np.where((M > 1e6) & (phm > 10**(13)) & (central == 1))

Mdot_filtered = Mdot[indices]
M_filtered = M[indices]
subID_filtered = subID[indices]

Mdot_edd = (4 * np.pi * G * M_filtered * mp) / (e_r * c * sigma_t) / 3.16887646e-8  # M_sun/yr

edd_ratio = Mdot_filtered / Mdot_edd

efficiency = 10 * edd_ratio * e_r
efficiency[edd_ratio > 0.1] = e_r #/(1-e_r)
Lbol = efficiency * Mdot_filtered * (1.989e+30 / 3.154e+7) * c**2 * 1e+7         #ergs/s
#Lbol1 = e_r * Mdot_filtered * (1.989e+30 / 3.154e+7) * c**2 * 1e+7         #ergs/s

#threshold = 0.1
threshold = np.minimum(0.002 * (M_filtered / 1e8)**2, 0.1)

Lcut = 1e41

sa_agn = subID_filtered[np.where((edd_ratio > threshold) & (Lbol > Lcut))]
sa_non = subID_filtered[np.where((edd_ratio > threshold) & (Lbol <= Lcut))]
sn_agn = subID_filtered[np.where((edd_ratio <= threshold) & (Lbol > Lcut))]
sn_non = subID_filtered[np.where((edd_ratio <= threshold) & (Lbol <= Lcut))]
'''
sa_agn = subID_filtered[np.where((edd_ratio > threshold) & (Lbol1 > Lcut))]
sa_non = subID_filtered[np.where((edd_ratio > threshold) & (Lbol1 <= Lcut))]
sn_agn = subID_filtered[np.where((edd_ratio <= threshold) & (Lbol1 > Lcut))]
sn_non = subID_filtered[np.where((edd_ratio <= threshold) & (Lbol1 <= Lcut))]
'''
ha_agn = phid[sa_agn]
ha_non = phid[sa_non]
hn_agn = phid[sn_agn]
hn_non = phid[sn_non]

np.savez("accretor_agn100_mine_hm5.npz", ha_agn = ha_agn, ha_non = ha_non, hn_agn = hn_agn, hn_non = hn_non)

# Print stats
total = len(subID_filtered)
print("Total halos:", total)
print("High accretor Central AGNs:", len(ha_agn))
print("High accretor Central Non-AGNs:", len(ha_non))
print("Non-accretor Central AGNs:", len(hn_agn))
print("Non-accretor Central Non-AGNs:", len(hn_non))

working
Total halos: 70
High accretor Central AGNs: 0
High accretor Central Non-AGNs: 0
Non-accretor Central AGNs: 38
Non-accretor Central Non-AGNs: 32
